In [ ]:
# BLOCK 1

#############################################################################################################################################################
#############################################################################################################################################################

number_of_rows = 200            
number_of_columns = 200       
dxy = 100                      

#############################################################################################################################################################
#############################################################################################################################################################

import numpy as np
from landlab import RasterModelGrid, imshow_grid
from landlab.components import StreamPowerEroder, LinearDiffuser, FlowAccumulator, ChannelProfiler, SteepnessFinder, ChiFinder
from matplotlib import pyplot as plt
import os
import os.path
from os import path
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore")
mg = RasterModelGrid((number_of_rows, number_of_columns), dxy)
np.random.seed(0)                                      
mg_noise = np.random.rand(mg.number_of_nodes)/1000.    
zr = mg.add_zeros('node', 'topographic__elevation')
zr += mg_noise

In [ ]:
# BLOCK 2

#############################################################################################################################################################
#############################################################################################################################################################

East = 4
North = 4
West = 1
South = 4

#############################################################################################################################################################
#############################################################################################################################################################

mg.set_status_at_node_on_edges(right=East, top=North, left=West, bottom=South)

In [ ]:
# BLOCK 3

#############################################################################################################################################################
#############################################################################################################################################################

Ksp_1 = 2E-4
Khs_1 = 1E-3

Ksp_2 = 1E-4
Khs_2 = 1E-3
K_Edge_2 = 10000

Ksp_3 = 0
Khs_3 = 0
K_Edge_3 = 0

Ksp_4 = 0
Khs_4 = 0
K_Edge_4 = 0

Ksp_5 = 0
Khs_5 = 0
K_Edge_5 = 0

m_sp = 0.5
n_sp = 1

#############################################################################################################################################################
#############################################################################################################################################################

Ksp = np.ones(mg.number_of_nodes) * Ksp_1                                                   
Ksp[np.where(mg.node_x<K_Edge_2)] = Ksp_2 
Ksp[np.where(mg.node_x<K_Edge_3)] = Ksp_3
Ksp[np.where(mg.node_x<K_Edge_4)] = Ksp_4
Ksp[np.where(mg.node_x<K_Edge_5)] = Ksp_5
Khs = np.ones(mg.number_of_nodes) * Khs_1                                                   
Khs[np.where(mg.node_x<K_Edge_2)] = Khs_2 
Khs[np.where(mg.node_x<K_Edge_3)] = Khs_3
Khs[np.where(mg.node_x<K_Edge_4)] = Khs_4
Khs[np.where(mg.node_x<K_Edge_5)] = Khs_5

In [ ]:
# BLOCK 4

#############################################################################################################################################################
#############################################################################################################################################################

U_1 = 5E-3

U_2 = 1E-3
U_Edge_2 = 10000

U_3 = 0
U_Edge_3 = 0

U_4 = 0
U_Edge_4 = 0

U_5 = 0
U_Edge_5 = 0

#############################################################################################################################################################
#############################################################################################################################################################

U = np.ones(mg.number_of_nodes) * U_1                                                   
U[np.where(mg.node_x<U_Edge_2)] = U_2 
U[np.where(mg.node_x<U_Edge_3)] = U_3
U[np.where(mg.node_x<U_Edge_4)] = U_4
U[np.where(mg.node_x<U_Edge_5)] = U_5

In [ ]:
# BLOCK 5

#############################################################################################################################################################
#############################################################################################################################################################

total_time = 0 
Plot_Ticker = 0
Export_DEM_Ticker = 0

#############################################################################################################################################################
#############################################################################################################################################################

In [ ]:
# BLOCK 6

#############################################################################################################################################################
#############################################################################################################################################################

dt = 100          
tmax = 2E5        

#############################################################################################################################################################
#############################################################################################################################################################

t = np.arange(0, tmax, dt) 

In [ ]:
# BLOCK 7

#############################################################################################################################################################
#############################################################################################################################################################

Plot_interval = 2E4
Export_DEM_Interval = 1000
number_of_watersheds = 5
min_drainage_area = 1000000

Directory = '/Users/Chris/Documents/'
DEM_Image = True
Slope_Area = True
Channel_Profile = True
Channel_Map = True
Ksn_Profile = True
Ksn_Map = True
Chi_Profile = True
Chi_Map = False
Erosion_Rate_Profile = False
Erosion_Rate_Map = False
DZ_DT_Profile = False
DZ_DT_Map = False
Ksp_Profile = True
Ksp_Map = False

#############################################################################################################################################################
#############################################################################################################################################################

In [ ]:
# BLOCK 8

frr = FlowAccumulator(mg, flow_director='D8') 
spr = StreamPowerEroder(mg, K_sp=Ksp, m_sp=m_sp, n_sp=n_sp)
dfn = LinearDiffuser(mg, linear_diffusivity=Khs)
for ti in t:
    zr[mg.core_nodes] += U[mg.core_nodes]*dt            
    dfn.run_one_step(dt)                                
    frr.run_one_step()                                  
    spr.run_one_step(dt)                                
    total_time += dt                                    
    print(total_time)
    Plot_Ticker += dt
    Export_DEM_Ticker += dt
    if Plot_Ticker == Plot_interval:
        if total_time == Plot_Ticker:
            prf = ChannelProfiler(mg, number_of_watersheds=number_of_watersheds, main_channel_only=True, minimum_channel_threshold=min_drainage_area)
            sf = SteepnessFinder(mg, reference_concavity=m_sp/n_sp, min_drainage_area=min_drainage_area)
            cf = ChiFinder(mg, reference_concavity=m_sp/n_sp, min_drainage_area=min_drainage_area)
            if path.exists(str(Directory)+'/TerrainSandbox') == False:
                os.mkdir(str(Directory)+'/TerrainSandbox')
    
        ##Figure 1###########################################################################
        if DEM_Image == True:
            if path.exists(str(Directory)+'/TerrainSandbox/DEM_Image') == False:
                os.mkdir(str(Directory)+'/TerrainSandbox/DEM_Image')
            plt.ioff()
            fig = plt.figure(1)         
            imshow_grid(mg, 'topographic__elevation', grid_units=('m', 'm'), cmap='terrain', allow_colorbar=True)
            title_text = '$Year$='+str(total_time)  
            plt.title(title_text)
            plt.tight_layout()
            fig.savefig(str(Directory)+'/TerrainSandbox/DEM_Image/'+str(total_time)+'.pdf',  format='pdf', dpi=300)
            plt.close(fig)
        ##Figure 1###########################################################################
    
        ##Figure 1###########################################################################
        if Slope_Area == True:
            prf.run_one_step()
            if path.exists(str(Directory)+'/TerrainSandbox/Slope_Area') == False:
                os.mkdir(str(Directory)+'/TerrainSandbox/Slope_Area')
            plt.ioff()
            fig = plt.figure(2)
            for i, outlet_id in enumerate(prf.data_structure):
                for j, segment_id in enumerate(prf.data_structure[outlet_id]):
                    if j == 0:
                        label = "channel {i}".format(i=i + 1)
                    else:
                        label = '_nolegend_'
                    segment = prf.data_structure[outlet_id][segment_id]
                    profile_ids = segment["ids"]
                    color = segment["color"]
                    plt.loglog(mg.at_node["drainage_area"][profile_ids], mg.at_node["topographic__steepest_slope"][profile_ids], '.', color=color, label=label)
            plt.legend(loc="lower left")
            plt.xlabel("drainage area (m^2)")
            plt.ylabel("channel slope [m/m]")
            title_text = '$Year$='+str(total_time)  
            plt.title(title_text)
            plt.grid(linestyle='--')
            plt.tight_layout()
            fig.savefig(str(Directory)+'/TerrainSandbox/Slope_Area/'+str(total_time)+'.pdf',  format='pdf', dpi=300)
            plt.close(fig)
        ##Figure 1###########################################################################
    
        ##Figure 1###########################################################################
        if Channel_Profile == True:
            prf.run_one_step()
            if path.exists(str(Directory)+'/TerrainSandbox/Channel_Profile') == False:
                os.mkdir(str(Directory)+'/TerrainSandbox/Channel_Profile')
            plt.ioff()
            fig = plt.figure(3)
            prf.plot_profiles(field='topographic__elevation', xlabel='distance upstream (m)', ylabel='elevation (m)')
            title_text = '$Year$='+str(total_time)
            plt.title(title_text)
            plt.grid(linestyle='--')
            plt.tight_layout()
            fig.savefig(str(Directory)+'/TerrainSandbox/Channel_Profile/'+str(total_time)+'.pdf',  format='pdf', dpi=300)
            plt.close(fig)
        ##Figure 1###########################################################################
        
        ##Figure 1###########################################################################
        if Channel_Map == True:
            prf.run_one_step()
            if path.exists(str(Directory)+'/TerrainSandbox/Channel_Map') == False:
                os.mkdir(str(Directory)+'/TerrainSandbox/Channel_Map')
            plt.ioff()
            fig = plt.figure(4)
            prf.plot_profiles_in_map_view()
            title_text = '$Year$='+str(total_time)
            plt.title(title_text)
            plt.tight_layout()
            fig.savefig(str(Directory)+'/TerrainSandbox/Channel_Map/'+str(total_time)+'.pdf',  format='pdf', dpi=300)
            plt.close(fig)
        ##Figure 1###########################################################################
        
        ##Figure 1###########################################################################
        if Ksn_Profile == True:
            prf.run_one_step()
            sf.calculate_steepnesses()
            if path.exists(str(Directory)+'/TerrainSandbox/Ksn_Profile') == False:
                os.mkdir(str(Directory)+'/TerrainSandbox/Ksn_Profile')
            plt.ioff()
            fig = plt.figure(5)
            for i, outlet_id in enumerate(prf.data_structure):
                for j, segment_id in enumerate(prf.data_structure[outlet_id]):
                    if j == 0:
                        label = "channel {i}".format(i=i + 1)
                    else:
                        label = '_nolegend_'
                    segment = prf.data_structure[outlet_id][segment_id]
                    profile_ids = segment["ids"]
                    distance_upstream = segment["distances"]
                    color = segment["color"]
                    plt.plot(distance_upstream, mg.at_node["channel__steepness_index"][profile_ids], 'x', color=color, label=label)
            plt.xlabel("distance upstream (m)")
            plt.ylabel("steepness index")
            plt.legend(loc="upper left")
            title_text = '$Year$='+str(total_time)  
            plt.title(title_text)
            plt.grid(linestyle='--')
            plt.tight_layout()
            fig.savefig(str(Directory)+'/TerrainSandbox/Ksn_Profile/'+str(total_time)+'.pdf',  format='pdf', dpi=300)
            plt.close(fig)
        ##Figure 1###########################################################################
        
        ##Figure 1###########################################################################
        if Ksn_Map == True:
            prf.run_one_step()
            sf.calculate_steepnesses()
            if path.exists(str(Directory)+'/TerrainSandbox/Ksn_Map') == False:
                os.mkdir(str(Directory)+'/TerrainSandbox/Ksn_Map')
            plt.ioff()
            fig = plt.figure(6)
            imshow_grid(mg, "channel__steepness_index", grid_units=("m", "m"), var_name="Steepness index ", cmap="jet")
            title_text = '$Year$='+str(total_time)  
            plt.title(title_text)
            plt.tight_layout()
            fig.savefig(str(Directory)+'/TerrainSandbox/Ksn_Map/'+str(total_time)+'.pdf',  format='pdf', dpi=300)
            plt.close(fig)
        ##Figure 1###########################################################################
        
        ##Figure 1###########################################################################
        if Chi_Profile == True:
            prf.run_one_step()
            cf.calculate_chi()
            if path.exists(str(Directory)+'/TerrainSandbox/Chi_Profile') == False:
                os.mkdir(str(Directory)+'/TerrainSandbox/Chi_Profile')
            plt.ioff()
            fig = plt.figure(7)
            for i, outlet_id in enumerate(prf.data_structure):
                for j, segment_id in enumerate(prf.data_structure[outlet_id]):
                    if j == 0:
                        label = "channel {i}".format(i=i + 1)
                    else:
                        label = '_nolegend_'
                    segment = prf.data_structure[outlet_id][segment_id]
                    profile_ids = segment["ids"]
                    color = segment["color"]
                    plt.plot(mg.at_node["channel__chi_index"][profile_ids], mg.at_node["topographic__elevation"][profile_ids], color=color, label=label)
            plt.xlabel("chi index (m)")
            plt.ylabel("elevation (m)")
            plt.legend(loc="lower right")
            title_text = '$Year$='+str(total_time)  
            plt.title(title_text)
            plt.grid(linestyle='--')
            plt.tight_layout()
            fig.savefig(str(Directory)+'/TerrainSandbox/Chi_Profile/'+str(total_time)+'.pdf',  format='pdf', dpi=300)
            plt.close(fig)
        
        
        
        
        
        
        
        
        
        
        
        
        
        ##Figure 1###########################################################################
        if Ksp_Profile == True:
            prf.run_one_step()
            if path.exists(str(Directory)+'/TerrainSandbox/Ksp_Profile') == False:
                os.mkdir(str(Directory)+'/TerrainSandbox/Ksp_Profile')
            plt.ioff()
            fig = plt.figure(117)
            prf.plot_profiles(field=Ksp, xlabel='distance upstream (m)', ylabel='Ksp')
            title_text = '$Year$='+str(total_time) 
            plt.title(title_text)
            plt.grid(linestyle='--')
            plt.tight_layout()
            fig.savefig(str(Directory)+'/TerrainSandbox/Ksp_Profile/'+str(total_time)+'.pdf',  format='pdf', dpi=300)
            plt.close(fig)
        ##Figure 1###########################################################################
        
        
        
        
        
        
        Plot_Ticker = 0